In [1]:
# Nov 20th, 2020
# I could not use dask here due to an error with metadata
# I'll move on for now but I could find the issue later

# February 12st, 2020
# This script compares classification in each classification system
# Second level matches specifications of Nemet and Johnson 2012

In [2]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import glob
import graphviz

file_list=glob.glob("data/citation/*")
dfs = [pd.read_parquet(f, columns=['patent_id']) for f in file_list]

In [3]:
df=dfs[0]

In [4]:
df.head()

,patent_id
citation_id,
4875247,5354551
D519335,D674253
1963218,D809697
7207426,D808172
6642945,8683318


In [5]:
class_df=pd.read_parquet('data/wipo.parquet.gz')
df=df.merge(class_df, right_index=True, left_index=True)

In [6]:
df=df.reset_index().set_index('patent_id') #reset_index avoid losing citation_id
df=df.merge(class_df, suffixes=['_citing', ''], right_index=True, left_index=True)

In [7]:
df

,index,wipo_field_id_citing,wipo_sector_id_citing,wipo_field_id,wipo_sector_id
patent_id,,,,,
10000001,9296144,29,Mechanical Eng,12,Instruments
10000002,4617207,09,Instruments,32,Mechanical Eng
10000004,6726363,25,Mechanical Eng,29,Mechanical Eng
10000007,3940227,29,Mechanical Eng,29,Mechanical Eng
10000007,5046349,26,Mechanical Eng,29,Mechanical Eng
...,...,...,...,...,...
9999989,7875192,31,Mechanical Eng,29,Mechanical Eng
9999989,8057915,29,Mechanical Eng,29,Mechanical Eng
9999989,8177541,29,Mechanical Eng,29,Mechanical Eng


In [8]:
# class_df=dd.read_parquet('data/wipo.parquet.gz')

for i, df in enumerate(dfs):
#     df=element.compute()
#     class_df=class_df.compute()
    df=df.merge(class_df, right_index=True, left_index=True)
    df=df.reset_index().set_index('patent_id') #reset_index avoid losing citation_id
    df=df.merge(class_df, suffixes=['_citing', ''], right_index=True, left_index=True)
    df['external']=df['wipo_field_id_citing']==df['wipo_field_id']    
    df['far_external']=df['wipo_sector_id_citing']==df['wipo_sector_id']    
    if i==0:
        output=df.groupby(df.index).sum()
    else:
        output=pd.concat([output, df.groupby(df.index).sum()]) # following Nemet and Johson, this variable is relevant for citations made

In [9]:
output.head()

,external,far_external
patent_id,,
10000001,0,0
10000002,0,0
10000004,0,1
10000007,1,6
10000008,1,1


In [14]:
output=output.groupby(output.index).sum()

In [15]:
# df.isnull().sum()
# df.set_index('uuid', inplace=True)
output.to_parquet('data/int_ext_cit.parquet.gz', compression='gzip')